# MobileNetV2

This notebook trains the MobileNetV2 model.

## Part 1 - Training the Model

### Importing the libraries

In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
sys.path.append('../')  

In [2]:
import sys
import os
import numpy as np
import json
import shutil
import keras
import tensorflow as tf

import logging
logging.basicConfig(level=logging.INFO)

from tornet.data.loader import get_dataloader
from tornet.data.preprocess import get_shape
from tornet.data.constants import ALL_VARIABLES
from tornet.models.keras.losses import mae_loss
from tornet.models.keras.mobilenet import build_model
from tornet.metrics.keras import metrics as tfm
from tornet.utils.general import make_exp_dir, make_callback_dirs


### Setting Up Environment Variables

In [3]:
# os.environ['KERAS_BACKEND']='tensorflow' # set to 'tensorflow', 'torch' or 'jax' (installs required)
os.environ['TORNET_ROOT'] = '../dataset'
EXP_DIR=os.environ.get('EXP_DIR','.')
DATA_ROOT=os.environ['TORNET_ROOT']
logging.info('TORNET_ROOT='+DATA_ROOT)


INFO:root:TORNET_ROOT=../dataset


### Setting Up CNN Configuration

In [4]:
DEFAULT_CONFIG={
    'epochs':10,
    'input_variables': ['DBZ',
               'VEL',
               'RHOHV',
               'ZDR',
               'WIDTH'],
    'train_years':list(range(2021,2022)),
    'val_years':list(range(2013,2014)),
    'batch_size':16,
    'model':'mobilenet',
    'start_filters':48,
    'learning_rate':1e-4,
    'decay_steps':1386,
    'decay_rate':0.958,
    'l2_reg':1e-5,
    'wN':1.0,
    'w0':1.0,
    'w1':1.0,
    'w2':2.0,
    'wW':0.5,
    'label_smooth':0,
    'loss':'cce',
    'head':'maxpool',
    'exp_name':'tornet-mobilenet',
    'exp_dir':EXP_DIR,
    'dataloader':"keras",
    'dataloader_kwargs': {}
}

### Building the Model

In [5]:
def train_keras_mobilenet_model(config):
    # Gather all hyperparameters
    epochs = config.get('epochs')
    batch_size = config.get('batch_size')
    start_filters = config.get('start_filters')
    learning_rate = config.get('learning_rate')
    decay_steps = config.get('decay_steps')
    decay_rate = config.get('decay_rate')
    l2_reg = config.get('l2_reg')
    wN = config.get('wN')
    w0 = config.get('w0')
    w1 = config.get('w1')
    w2 = config.get('w2')
    wW = config.get('wW')
    head = config.get('head')
    label_smooth = config.get('label_smooth')
    loss_fn = config.get('loss')
    input_variables = config.get('input_variables')
    exp_name = config.get('exp_name')
    exp_dir = config.get('exp_dir')
    train_years = config.get('train_years')
    val_years = config.get('val_years')
    dataloader = config.get('dataloader')
    dataloader_kwargs = config.get('dataloader_kwargs')
    input_variables = config.get('input_variables')

    logging.info(f"Using {tf.keras.backend.backend()} backend")
    logging.info(f'Using {dataloader} dataloader')
    logging.info('Running with config:')
    logging.info(config)

    weights = {'wN': wN, 'w0': w0, 'w1': w1, 'w2': w2, 'wW': wW}

    # Create data loaders
    dataloader_kwargs = {'select_keys': input_variables + ['range_folded_mask', 'coordinates']}
    ds_train = get_dataloader(dataloader, DATA_ROOT, train_years,
                              "train", batch_size, weights, **dataloader_kwargs)
    ds_val = get_dataloader(dataloader, DATA_ROOT, val_years,
                            "train", batch_size, weights, **dataloader_kwargs)

    in_shapes = (120, 240, 2)
    c_shapes = (120, 240, 2)
    nn = build_model(shape=in_shapes,
                                   c_shape=c_shapes,
                                   start_filters=start_filters,
                                   l2_reg=l2_reg,
                                   input_variables=input_variables,
                                   head=head)

    # Model setup
    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate, decay_steps, decay_rate, staircase=False, name="exp_decay")

    from_logits = True
    if loss_fn.lower() == 'cce':
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=from_logits,
                                                  label_smoothing=label_smooth)
    elif loss_fn.lower() == 'hinge':
        loss = tf.keras.losses.Hinge()  # automatically converts labels to -1,1
    elif loss_fn.lower() == 'mae':
        def loss(yt, yp): return mae_loss(yt, yp)
    else:
        raise RuntimeError('unknown loss %s' % loss_fn)

    opt = tf.keras.optimizers.Adam(learning_rate=lr)

    # Compute various metrics while training
    metrics_list = [keras.metrics.AUC(from_logits=from_logits, name='AUC', num_thresholds=2000),
               keras.metrics.AUC(from_logits=from_logits,
                                 curve='PR', name='AUCPR', num_thresholds=2000),
               tfm.BinaryAccuracy(from_logits, name='BinaryAccuracy'),
               tfm.TruePositives(from_logits, name='TruePositives'),
               tfm.FalsePositives(from_logits, name='FalsePositives'),
               tfm.TrueNegatives(from_logits, name='TrueNegatives'),
               tfm.FalseNegatives(from_logits, name='FalseNegatives'),
               tfm.Precision(from_logits, name='Precision'),
               tfm.Recall(from_logits, name='Recall'),
               tfm.F1Score(from_logits=from_logits, name='F1')]

    # Compile the model
    nn.compile(loss=loss,
               metrics=metrics_list,
               optimizer=opt,
               weighted_metrics=[])
    
    steps_per_epoch = len(ds_train) // batch_size
    print(f"steps_per_epoch: {steps_per_epoch}, len(ds_train): {len(ds_train)}, batch_size: {batch_size}")
    # FIT with ModelCheckpoint in callbacks
    callbacks = []  # Add other callbacks here if necessary
    nn.fit(ds_train, epochs=epochs, validation_data=ds_val, callbacks=callbacks, verbose=1)

    # print ds_train
    # print ds_val
    print(ds_train)
    return nn

# Ensure the TensorFlow graph is reset if running in a notebook
tf.keras.backend.clear_session()

### Training the MobileNetV2 model

In [6]:
mobilenet_model = train_keras_mobilenet_model(DEFAULT_CONFIG)
mobilenet_model.save('mobilenet_5feature_model.h5')

INFO:root:Using tensorflow backend
INFO:root:Using keras dataloader
INFO:root:Running with config:
INFO:root:{'epochs': 10, 'input_variables': ['DBZ', 'VEL', 'RHOHV', 'ZDR', 'WIDTH'], 'train_years': [2021], 'val_years': [2013], 'batch_size': 16, 'model': 'convnext', 'start_filters': 48, 'learning_rate': 0.0001, 'decay_steps': 1386, 'decay_rate': 0.958, 'l2_reg': 1e-05, 'wN': 1.0, 'w0': 1.0, 'w1': 1.0, 'w2': 2.0, 'wW': 0.5, 'label_smooth': 0, 'loss': 'cce', 'head': 'maxpool', 'exp_name': 'tornet_convnext', 'exp_dir': '.', 'dataloader': 'keras', 'dataloader_kwargs': {}}


steps_per_epoch: 74, len(ds_train): 1191, batch_size: 16
Epoch 1/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 953s 785ms/step - AUC: 0.5185 - AUCPR: 0.0730 - BinaryAccuracy: 0.9115 - F1: 0.0000e+00 - FalseNegatives: 674.4144 - FalsePositives: 52.9471 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - TrueNegatives: 8813.3193 - TruePositives: 0.0000e+00 - loss: 0.3176 - val_AUC: 0.5000 - val_AUCPR: 0.0941 - val_BinaryAccuracy: 0.9059 - val_F1: 0.0000e+00 - val_FalseNegatives: 329.0000 - val_FalsePositives: 0.0000e+00 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_TrueNegatives: 3169.0000 - val_TruePositives: 0.0000e+00 - val_loss: 0.4083
Epoch 2/10
1191/1191 ━━━━━━━━━━━━━━━━━━━━ 932s 780ms/step - AUC: 0.6537 - AUCPR: 0.1093 - BinaryAccuracy: 0.9322 - F1: 0.0000e+00 - FalseNegatives: 659.3389 - FalsePositives: 0.0000e+00 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - TrueNegatives: 8882.2441 - TruePositives: 0.0000e+00 - loss: 0.2702 - val_AUC: 0.5000 - val_AUCPR: 0.0941 - val_BinaryAccurac

In [7]:
mobilenet_model.save('mobilenet_5feature_model.keras')

### Evaluating the Model

In [8]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from tornet.metrics.keras import metrics as tfm
import logging


from tornet.data.keras.loader import KerasDataLoader


In [13]:
def get_subset_test_data(years, category):
    # Create test samples
    # Load full catalog and select EF 3+ tornadoes
    os.environ['TORNET_ROOT'] = '../dataset'
    data_root=os.environ['TORNET_ROOT']

    catalog_path = os.path.join(data_root,'catalog.csv')
    if not os.path.exists(catalog_path):
        raise RuntimeError('Unable to find catalog.csv at '+data_root)
            
    catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])

    catalog = catalog[(catalog.start_time.dt.year.isin(years)) & (catalog['category'].isin(category))]
    # catalog = catalog[(catalog.start_time.dt.year.isin([2021]))]

    ds_test = KerasDataLoader(data_root=data_root,
                            data_type='test',
                            random_state=1234,
                            catalog=catalog,
                            batch_size = 64, 
                            use_multiprocessing = True)

    return ds_test

def evalate_model(model, ds_test):
    # Evaluate the model
    model = keras.models.load_model('mobilenet_5feature_model.keras')
    # Compute various metrics
    from_logits = True
    metrics = [keras.metrics.AUC(from_logits=from_logits, name='AUC', num_thresholds=2000),
               keras.metrics.AUC(from_logits=from_logits,
                                 curve='PR', name='AUCPR', num_thresholds=2000),
               tfm.BinaryAccuracy(from_logits, name='BinaryAccuracy'),
               tfm.TruePositives(from_logits, name='TruePositives'),
               tfm.FalsePositives(from_logits, name='FalsePositives'),
               tfm.TrueNegatives(from_logits, name='TrueNegatives'),
               tfm.FalseNegatives(from_logits, name='FalseNegatives'),
               tfm.Precision(from_logits, name='Precision'),
               tfm.Recall(from_logits, name='Recall'),
               tfm.F1Score(from_logits=from_logits, name='F1')]
    model.compile(metrics=metrics)

    scores = model.evaluate(ds_test)
    scores = {m.name: scores[k+1] for k, m in enumerate(metrics)}

    logging.info(scores)

## Test the MobileNetV2 model

In [14]:
# Load saved model
model_file = 'mobilenet_5feature_model.keras' # change to the model file path
mobilenet_model = keras.models.load_model(model_file,compile=False)

In [15]:
# Build a test set
ds_test = get_dataloader("keras", DATA_ROOT, [2021], "test", 128, {}, select_keys=ALL_VARIABLES + ['range_folded_mask', 'coordinates'])

evalate_model(mobilenet_model, ds_test)


c:\Users\zhouj\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 166 variables whereas the saved optimizer has 329 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


34/34 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step - AUC: 0.4459 - AUCPR: 0.0384 - BinaryAccuracy: 0.9553 - F1: 0.0514 - FalseNegatives: 88.1429 - FalsePositives: 17.6286 - Precision: 0.1552 - Recall: 0.0311 - TrueNegatives: 2187.6572 - TruePositives: 2.1143 - loss: 0.0075


INFO:root:{'AUC': 0.44930654764175415, 'AUCPR': 0.03507734835147858, 'BinaryAccuracy': 0.9529053568840027, 'TruePositives': 3.0, 'FalsePositives': 31.0, 'TrueNegatives': 4064.0, 'FalseNegatives': 170.0, 'Precision': 0.0882352963089943, 'Recall': 0.017341040074825287, 'F1': 0.028985481709241867}
